<a href="https://colab.research.google.com/github/velovskam/LinkPrediction-NodeClassification/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I-дел Link Prediction

Инсталација на OGB со Python package manager pip со цел да го инсталираме податочното множество ogbl-ppa.

In [ ]:
pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=3d6b56fce913dd81cb6b1b6b232eafb76bf2e41aed4a9745322e6a799972431c
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 30.6 MB/s eta 0:00:00


In [ ]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=8acdbcb088f5247841f17039df144d4ae7e23b0acf846790ceb35b9f2c0d5b8c
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


Load library

In [ ]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import torch.optim as optim

from ogb.graphproppred import DglGraphPropPredDataset
from dgl.dataloading import GraphDataLoader
from ogb.linkproppred import DglLinkPropPredDataset
from torch_geometric.nn import SAGEConv
from sklearn.metrics import roc_auc_score
from dgl.nn import SAGEConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Load data

In [ ]:
dataset = DglLinkPropPredDataset(name='ogbl-ppa')

Downloaded 0.38 GB: 100%|██████████| 388/388 [00:14<00:00, 27.60it/s]


Extracting dataset/ppassoc.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  5.40it/s]


Saving...


In [ ]:
num_edges = dataset[0].number_of_edges()
print(f"Number of edges: {num_edges}")

Number of edges: 42463862


In [ ]:
#го делиме гарикот на повеќе групи за тренирање, тестирање и валидација
split_edge = dataset.get_edge_split()

# Retrieve the training edges
train_edges = split_edge['train']['edge'].numpy()
# Retrieve the validation edges
valid_edges = split_edge['valid']['edge'].numpy()
# Retrieve the test edges
test_edges = split_edge['test']['edge'].numpy()

print(split_edge['train'].keys())
print(split_edge['valid'].keys())
print(split_edge['test'].keys())

dict_keys(['edge'])
dict_keys(['edge', 'edge_neg'])
dict_keys(['edge', 'edge_neg'])


In [ ]:
g = dataset[0]
g
#пристап до првиот график

Graph(num_nodes=576289, num_edges=42463862,
      ndata_schemes={'feat': Scheme(shape=(58,), dtype=torch.int64)}
      edata_schemes={})

In [ ]:
dataset = dgl.data.CiteseerGraphDataset()
g= dataset[0]

Extracting file to /root/.dgl/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Link Prediction

In [ ]:
class DotProductPredictor(nn.Module):
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return graph.edata['score']

In [ ]:
k=100
def construct_negative_graph(graph, k):
    src, dst = graph.edges()

    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(), (len(src) * k,))
    return dgl.graph((neg_src, neg_dst), num_nodes=graph.num_nodes())

In [ ]:
in_features = 58
hidden_dim = 64
out_features = 1
num_epochs = 10
learning_rate = 0.01
batch_size = 64
k = 100

In [ ]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(SAGE, self).__init__()
        self.sage_conv = SAGEConv(in_feats, hidden_feats, aggregator_type='mean')
        self.fc = nn.Linear(hidden_feats, out_feats)

    def forward(self, g, x):
        h = self.sage_conv(g, x)
        h = self.fc(h)
        return h

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.sage = SAGE(in_features, hidden_features, out_features)
        self.pred = DotProductPredictor()
    def forward(self, g, neg_g, x):
        h = self.sage(g, x)
        return self.pred(g, h), self.pred(neg_g, h)

In [ ]:
def compute_auc(pos_score, neg_score):
    y_true = [1] * len(pos_score) + [0] * len(neg_score)
    y_scores = torch.cat([pos_score, neg_score]).cpu().numpy()
    return roc_auc_score(y_true, y_scores)

In [ ]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

Training

In [ ]:
def compute_loss(pos_score, neg_score):
    # Margin loss
    n_edges = pos_score.shape[0]
    return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

node_features = g.ndata['feat']
n_features = node_features.shape[1]
k = 100
loss_function = torch.nn.CrossEntropyLoss()
model = Model(n_features, 100, 100)
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    model.train()
    negative_graph = construct_negative_graph(g, k)
    pos_score, neg_score = model(g, negative_graph, node_features)
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f'Epoch [{epoch+1}/{num_epochs}]', loss.item())

print('Training finished.')

Epoch [1/10] 0.9997650384902954
Epoch [2/10] 0.998723566532135
Epoch [3/10] 0.9975135326385498
Epoch [4/10] 0.9959850311279297
Epoch [5/10] 0.993999719619751
Epoch [6/10] 0.9914907217025757
Epoch [7/10] 0.9882992506027222
Epoch [8/10] 0.9844083786010742
Epoch [9/10] 0.9796165227890015
Epoch [10/10] 0.973762571811676
Training finished.


II-дел Node Classification

In [ ]:
pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 927.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=1fd2283bb5481e0ff0146d6db021005a48d2c79206f90e2352e8f555b3484584
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 13.2 MB/s eta 0:00:00


Load library

In [ ]:
import dgl
import torch
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

from ogb.nodeproppred import DglNodePropPredDataset

Load data

In [ ]:
dataset = DglNodePropPredDataset(name='ogbn-arxiv')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:06<00:00, 13.20it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 2154.24it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00, 174.78it/s]

Saving...


In [ ]:
#го делиме гарикот на повеќе групи за тренирање, тестирање и валидација
split_idx = dataset.get_idx_split()
g, labels = dataset[0]
# get split labels
train_label = dataset.labels[split_idx['train']]
valid_label = dataset.labels[split_idx['valid']]
test_label = dataset.labels[split_idx['test']]

In [ ]:
graph = dataset[0]
graph

(Graph(num_nodes=169343, num_edges=1166243,
       ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
       edata_schemes={}),
 tensor([[ 4],
         [ 5],
         [28],
         ...,
         [10],
         [ 4],
         [ 1]]))

Node Classification

In [ ]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, dropout_prob=0.5):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.dropout(h)  # Apply dropout
        h = self.conv2(graph, h)
        return h


In [ ]:
dataset = dgl.data.CiteseerGraphDataset()
graph = dataset[0]
graph

Extracting file to /root/.dgl/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(3703,), dtype=torch.float32)}
      edata_schemes={})

In [ ]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [ ]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [ ]:
n_feats = 128
hid_feats = 200
n_labels = 40
num_epochs=100

Training

In [ ]:
model = SAGE(in_feats=n_features, hid_feats=200, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay = 5e-4)


for epoch in range(num_epochs):
    model.train()
    # forward propagation by using all nodes
    logits = model(graph, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f'Epoch [{epoch+1}/{num_epochs}]',loss.item(), f'Accuracy: {acc * 100:.2f}%')

print('Training finished.')

Epoch [1/100] 3.700935125350952 Accuracy: 0.00%
Epoch [2/100] 3.6745142936706543 Accuracy: 1.00%
Epoch [3/100] 3.653712511062622 Accuracy: 10.20%
Epoch [4/100] 3.628223180770874 Accuracy: 23.00%
Epoch [5/100] 3.6029293537139893 Accuracy: 27.40%
Epoch [6/100] 3.5807981491088867 Accuracy: 29.60%
Epoch [7/100] 3.557154417037964 Accuracy: 30.80%
Epoch [8/100] 3.529545783996582 Accuracy: 32.00%
Epoch [9/100] 3.503796100616455 Accuracy: 32.40%
Epoch [10/100] 3.4699461460113525 Accuracy: 33.80%
Epoch [11/100] 3.437971353530884 Accuracy: 35.60%
Epoch [12/100] 3.3989298343658447 Accuracy: 37.20%
Epoch [13/100] 3.3664984703063965 Accuracy: 39.80%
Epoch [14/100] 3.3295505046844482 Accuracy: 42.00%
Epoch [15/100] 3.2847697734832764 Accuracy: 43.40%
Epoch [16/100] 3.2414157390594482 Accuracy: 45.40%
Epoch [17/100] 3.197202205657959 Accuracy: 46.60%
Epoch [18/100] 3.1551849842071533 Accuracy: 48.40%
Epoch [19/100] 3.11104679107666 Accuracy: 50.20%
Epoch [20/100] 3.0566329956054688 Accuracy: 51.00%
E

Evaluation + Testing

In [ ]:
# After training loop
model.eval()
val_accuracy = evaluate(model, graph, node_features, node_labels, valid_mask)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

Validation Accuracy: 70.00%


In [ ]:
test_accuracy = evaluate(model, graph, node_features, node_labels, test_mask)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 68.30%


III-дел CORA DATASET

In [ ]:
!pip install dgl-cu110 dglgo -f https://data.dgl.ai/wheels/repo.html &>/dev/null

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import itertools
from tqdm import tqdm

import dgl
from dgl.data import DGLDataset
import dgl.nn as dglnn
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DATA LOADING

In [ ]:
content_path = "/content/drive/MyDrive/CORA/cora.content"
cite_path = "/content/drive/MyDrive/CORA/cora.cites"

READING DATA

In [ ]:
with open(content_path, "r") as fp:
    contents = fp.readlines()
with open(cite_path, "r") as fp:
    cites = fp.readlines()

PARSING

In [ ]:
contents = np.array([np.array(l.strip().split("\t")) for l in contents])
paper_list, feat_list, label_list = np.split(contents, [1,-1], axis=1)
paper_list, label_list = np.squeeze(paper_list), np.squeeze(label_list)

Data Dictionary Creation

In [ ]:
paper_dict = dict([(key, val) for val, key in enumerate(paper_list)])
labels = list(set(label_list))
label_dict = dict([(key, val) for val, key in enumerate(labels)])

Edge Generation

In [ ]:
cites = [i.strip().split("\t") for i in cites]
cites = np.array([[paper_dict[i[0]], paper_dict[i[1]]] for i in cites],
                 np.int64).T   # (2, edge)
cites = np.concatenate((cites, cites[::-1, :]), axis=1)  # (2, 2*edge) or (2, E)

Degree Calculation:

In [ ]:
_, degree_list = np.unique(cites[0,:], return_counts=True)

Preparing the input data for a graph-based machine learning

In [ ]:
node_num = len(paper_list)
feat_dim = feat_list.shape[1]
stat_dim = 32
num_class = len(labels)
T = 2

In [ ]:
feat_Matrix = torch.Tensor(feat_list.astype(np.float32))
X_Node, X_Quote = np.split(cites, 2, axis=0)
X_Node, X_Quote = torch.from_numpy(np.squeeze(X_Node)), \
                 torch.from_numpy(np.squeeze(X_Quote))
label_list = np.array([label_dict[i] for i in label_list])
label_list = torch.from_numpy(label_list)

In [ ]:
print('Node:', X_Node)
print('Quote:', X_Quote)
print('label:', label_list)
print('feat:', feat_Matrix)

Node: tensor([ 163,  163,  163,  ..., 2258, 1887, 1686])
Quote: tensor([ 402,  659, 1696,  ..., 1887, 1902,  837])
label: tensor([2, 1, 3,  ..., 5, 0, 2])
feat: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


Creating a DGL (Deep Graph Library) graph object and adding node features and labels to it

In [ ]:
g = dgl.graph((X_Node, X_Quote), num_nodes=node_num)
g.ndata['feat'] = feat_Matrix
g.ndata['label'] = label_list

Spliting the data

In [ ]:
n_nodes = node_num
n_train = int(n_nodes * 0.6)
n_val = int(n_nodes * 0.2)

In [ ]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[:n_train] = True
val_mask[n_train:n_train + n_val] = True
test_mask[n_train + n_val:] = True

In [ ]:
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask
g.ndata['test_mask'] = test_mask

In [ ]:
print('Graph summary')
print(g)
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Graph summary
Graph(num_nodes=2708, num_edges=10858,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})
Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([2, 1, 3,  ..., 5, 0, 2]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True])}
Edge features
{}


In [ ]:
from dgl.nn import GraphConv
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']

Training

In [ ]:
def train(g, model, epoch, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_val_acc = 0
    best_test_acc = 0

    for e in range(epoch):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 50 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

Create the model with given dimensions

In [ ]:
model = GCN(g.ndata['feat'].shape[1], 16, len(g.ndata['label'].unique()))
train(g, model, epoch = 500, lr = 0.001)

In epoch 0, loss: 1.947, val acc: 0.074 (best 0.074), test acc: 0.092 (best 0.092)
In epoch 50, loss: 1.318, val acc: 0.749 (best 0.749), test acc: 0.689 (best 0.689)
In epoch 100, loss: 0.772, val acc: 0.800 (best 0.800), test acc: 0.759 (best 0.759)
In epoch 150, loss: 0.489, val acc: 0.843 (best 0.843), test acc: 0.785 (best 0.783)
In epoch 200, loss: 0.351, val acc: 0.839 (best 0.845), test acc: 0.790 (best 0.794)
In epoch 250, loss: 0.274, val acc: 0.843 (best 0.847), test acc: 0.790 (best 0.792)
In epoch 300, loss: 0.225, val acc: 0.837 (best 0.847), test acc: 0.786 (best 0.792)
In epoch 350, loss: 0.191, val acc: 0.837 (best 0.847), test acc: 0.790 (best 0.792)
In epoch 400, loss: 0.165, val acc: 0.834 (best 0.847), test acc: 0.792 (best 0.792)
In epoch 450, loss: 0.144, val acc: 0.832 (best 0.847), test acc: 0.796 (best 0.792)
